In [1]:
import numpy as np
import pickle
import pandas as pd

In [2]:
original_auis_filename='/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/INTERSECT_AUI2ID.PICKLE'
new_auis_filename='/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/UNIQUE_AUI2ID.PICKLE'

original_auis = pickle.load(open(original_auis_filename, 'rb'))
new_auis = pickle.load(open(new_auis_filename, 'rb'))

In [3]:
original_auis = list(original_auis.keys())
new_auis = list(new_auis.keys())

In [6]:
pickle.dump(original_auis,open('../data/vishesh_dataset_original_auis.p','wb'))
pickle.dump(new_auis,open('../data/vishesh_dataset_new_auis.p','wb'))

In [ ]:
pickle.dump(original_auis[:1000],open('../data/vishesh_dataset_subset_original_auis.p','wb'))
pickle.dump(new_auis[:1000],open('../data/vishesh_dataset_subset_new_auis.p','wb'))